In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://securit

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Review-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [4]:
# vine_table dataframe from Deliverable 1
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [5]:
# Helpful reviews filter (total_votes >= 20)
filtered_df = vine_df.filter("total_votes>=20")
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [6]:
from pyspark.sql.types import FloatType
filtered_float_df = filtered_df.withColumn('helpful_votes', filtered_df['helpful_votes'].cast(FloatType()))
filtered_float_df = filtered_df.withColumn('total_votes', filtered_df['total_votes'].cast(FloatType()))

In [7]:
# Helpful_votes filter
helpful_df = filtered_float_df.filter("helpful_votes/total_votes >= .50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|       23.0|   N|                Y|
|R227GSNWI6BSZV|          1|           20|       20.0|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|       99.0|   N|                Y|
|R248FG65D76D5Y|          1|           42|       53.0|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|       32.0|   N|                Y|
| ROTIV4VYL31R4|          5|           26|       26.0|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|       25.0|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|       35.0|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|       84.0|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|      132.0|   N|     

In [8]:
# Vine members filter 
y_vine_df = helpful_df.filter(helpful_df.vine == 'Y')
y_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R184FOUNZZ7KO8|          5|           15|       20.0|   Y|                N|
| R82QWN2X2OCHB|          5|          176|      208.0|   Y|                N|
|R1UYHBYE6790BU|          5|           44|       53.0|   Y|                N|
|R2J3YLX1L4EH2B|          5|          299|      321.0|   Y|                N|
|R3QDI539WTXKE2|          5|           26|       32.0|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
# None-vine members filter
n_vine_df = helpful_df.filter(helpful_df.vine == 'N')
n_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|       23.0|   N|                Y|
|R227GSNWI6BSZV|          1|           20|       20.0|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|       99.0|   N|                Y|
|R248FG65D76D5Y|          1|           42|       53.0|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|       32.0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [13]:
# Potential bias
from pyspark.sql.functions import count

# Total reviews (Paid vs Unpaid)
paid_total = y_vine_df.count()
unpaid_total = n_vine_df.count()

print("Total Number of  helpful paid electronics reviews: %f" % paid_total)
print("Total Number of helpful unpaid electronics reviews: %f" % unpaid_total)

Total Number of  helpful paid electronics reviews: 1080.000000
Total Number of helpful unpaid electronics reviews: 49673.000000


In [14]:
# Total 5-star reviews (Paid, Unpaid )
paid_5star = y_vine_df.filter(y_vine_df['star_rating'] == 5)
unpaid_5star = n_vine_df.filter(n_vine_df['star_rating'] == 5)

print("Number of 5-star helpful paid electronics reviews: %f" % paid_5star.count())
print(" Number of 5-star helpful unpaid electronics reviews: %f" % unpaid_5star.count())

Number of 5-star helpful paid electronics reviews: 454.000000
 Number of 5-star helpful unpaid electronics reviews: 23043.000000


In [15]:
#  5-star reviews percentage (Paid, Unpaid)
paid_5star = paid_5star.count()
unpaid_5star = unpaid_5star.count()

paid_5star_percentage = (paid_5star/paid_total) * 100
unpaid_5star_percentage = (unpaid_5star/unpaid_total) * 100
print("Percentage of  5-star 5-star helpful paid electronics reviews: %f" % paid_5star_percentage)
print("Percentage of  5-star 5-star helpful unpaid electronics reviews: %f" % unpaid_5star_percentage)

Percentage of  5-star 5-star helpful paid electronics reviews: 42.037037
Percentage of  5-star 5-star helpful unpaid electronics reviews: 46.389387
